# Import

In [ ]:
#For Preprocessing
import re    # RegEx for removing non-letter characters
import sys
import pandas as pd
import os
import boto3
import collections
import json
import gensim
import time
import nltk
import html

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from io import StringIO
from collections import defaultdict
from datetime import datetime, timedelta
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.parse.malt import MaltParser
from nltk.corpus import words

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')
nltk.download('vader_lexicon')

# Recursively copy from S3 to Data folder

In [6]:
# !aws s3 cp --recursive s3://bigdatabuddies-glue-output ../Data

SyntaxError: invalid syntax (2817381454.py, line 1)

In [7]:
dfs = []
for file in os.listdir('../Data/'):
    if file.endswith('.csv'):
        print(file)
        df = pd.read_csv(os.path.join('../Data/', file))
        dfs.append(df)
concat_df = pd.concat(dfs, ignore_index=True)

run-1680588414414-part-r-00000.csv
run-1680588414414-part-r-00001.csv
run-1680588414414-part-r-00002.csv
run-1680588414414-part-r-00003.csv
run-1680588414414-part-r-00004.csv
run-1680588414414-part-r-00005.csv
run-1680588414414-part-r-00006.csv
run-1680588414414-part-r-00007.csv
run-1680588414414-part-r-00008.csv
run-1680588414414-part-r-00009.csv
run-1680588414414-part-r-00010.csv
run-1680588414414-part-r-00011.csv
run-1680588414414-part-r-00012.csv
run-1680588414414-part-r-00013.csv
run-1680588414414-part-r-00014.csv
run-1680588414414-part-r-00015.csv
run-1680588414414-part-r-00016.csv
run-1680588414414-part-r-00017.csv
run-1680588414414-part-r-00018.csv
run-1680588414414-part-r-00019.csv
run-1680588414414-part-r-00020.csv
run-1680588414414-part-r-00021.csv
run-1680588414414-part-r-00022.csv
run-1680588414414-part-r-00023.csv
run-1680588414414-part-r-00024.csv
run-1680588414414-part-r-00025.csv
run-1680588414414-part-r-00026.csv
run-1680588414414-part-r-00027.csv
run-1680588414414-pa

In [8]:
concat_df.to_csv("../Temp/concat.csv")

# Preprocess data

In [6]:
concat_df = pd.read_csv("../Temp/concat.csv")
concat_df.drop('Unnamed: 0', axis=1, inplace=True)
concat_df

C:\Users\jewel\AppData\Local\Temp\ipykernel_24612\4148428780.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  concat_df = pd.read_csv("../Temp/{}_{}_concat.csv".format(today_str, current_month))


,datetime,id,text,username
0,2022-12-01 23:59:58+00:00,1598466969141137408,The new ChatGPT model by OpenAI is astonishi...,askviable
1,2022-12-01 23:59:56+00:00,1598466963466252288,bitquark I am looking forward for a ChatGPT A...,mazen160
2,2022-12-01 23:59:37+00:00,1598466881761218560,i think i broke chatGPT it s been frozen for ...,ZoumanaCisse6
3,2022-12-01 23:59:26+00:00,1598466834776600576,YanoTomo aizu sniper yae AI Ch...,papakichi111
4,2022-12-01 23:59:20+00:00,1598466812198387712,I like ChatGPT as much as the next guy but th...,DennisLibre
...,...,...,...,...
5663206,2022-11-09 00:00:50+00:00,1590132269255114753,Reduce cost and improve query performance with...,tweepy20220507
5663207,2022-11-09 00:00:36+00:00,1590132210715303936,Enable cross account queries on AWS CloudTrail...,tweepy20220507
5663208,2022-11-09 00:00:12+00:00,1590132106260353024,If you want to generate more leads create con...,PirenneSalvor
5663209,2022-11-09 00:00:03+00:00,1590132069069619200,AnnaTynan Yes bc we share an inbox I am lu...,ceciliaclyra


In [14]:
def decode_text(text):
    decoded_text = text.encode('ascii', 'ignore').decode('utf-8')

    # decode HTML entities
    decoded_html = html.unescape(decoded_text)
    return ''.join([word for word in decoded_html if word.isprintable()])

def remove_mentions(text):
    return re.sub("@[A-Za-z0-9_]+","", text)

def remove_stopwords(words_list):
    stop_list = stopwords.words("english")
    stop_list.append("filler")
    stop_list.extend(["html", "http", "https", "www", "gt", "amp", "com", "nbsp", "em", "en", "lt", "quot"])
    clean_sentence = ''
    for word in words_list:
        if word not in stop_list:
            clean_sentence += word + ' '
    return clean_sentence

def clean_original_text(text):
    text = str(text)
    text = text.lower()
    decoded_sentence = decode_text(text)
    decoded_tokens = decoded_sentence.split()
    clean_sentence = remove_stopwords(decoded_tokens)

    return clean_sentence

In [15]:
concat_df['clean_text'] = concat_df['text'].apply(clean_original_text)

In [16]:
concat_df.to_csv("../Temp/preprocessed.csv", index=False)

# Recursively copy to Data folder

In [6]:
# !aws s3 cp --recursive s3://bigdatabuddies-glue-output ../Data

SyntaxError: invalid syntax (2817381454.py, line 1)

In [7]:
dfs = []
for file in os.listdir('../Data/'):
    if file.endswith('.csv'):
        print(file)
        df = pd.read_csv(os.path.join('../Data/', file))
        dfs.append(df)
concat_df = pd.concat(dfs, ignore_index=True)

run-1680588414414-part-r-00000.csv
run-1680588414414-part-r-00001.csv
run-1680588414414-part-r-00002.csv
run-1680588414414-part-r-00003.csv
run-1680588414414-part-r-00004.csv
run-1680588414414-part-r-00005.csv
run-1680588414414-part-r-00006.csv
run-1680588414414-part-r-00007.csv
run-1680588414414-part-r-00008.csv
run-1680588414414-part-r-00009.csv
run-1680588414414-part-r-00010.csv
run-1680588414414-part-r-00011.csv
run-1680588414414-part-r-00012.csv
run-1680588414414-part-r-00013.csv
run-1680588414414-part-r-00014.csv
run-1680588414414-part-r-00015.csv
run-1680588414414-part-r-00016.csv
run-1680588414414-part-r-00017.csv
run-1680588414414-part-r-00018.csv
run-1680588414414-part-r-00019.csv
run-1680588414414-part-r-00020.csv
run-1680588414414-part-r-00021.csv
run-1680588414414-part-r-00022.csv
run-1680588414414-part-r-00023.csv
run-1680588414414-part-r-00024.csv
run-1680588414414-part-r-00025.csv
run-1680588414414-part-r-00026.csv
run-1680588414414-part-r-00027.csv
run-1680588414414-pa

In [8]:
concat_df.to_csv("../Temp/concat.csv")

# Preprocess data

In [6]:
concat_df = pd.read_csv("../Temp/concat.csv")
concat_df.drop('Unnamed: 0', axis=1, inplace=True)
concat_df

C:\Users\jewel\AppData\Local\Temp\ipykernel_24612\4148428780.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  concat_df = pd.read_csv("../Temp/{}_{}_concat.csv".format(today_str, current_month))


,datetime,id,text,username
0,2022-12-01 23:59:58+00:00,1598466969141137408,The new ChatGPT model by OpenAI is astonishi...,askviable
1,2022-12-01 23:59:56+00:00,1598466963466252288,bitquark I am looking forward for a ChatGPT A...,mazen160
2,2022-12-01 23:59:37+00:00,1598466881761218560,i think i broke chatGPT it s been frozen for ...,ZoumanaCisse6
3,2022-12-01 23:59:26+00:00,1598466834776600576,YanoTomo aizu sniper yae AI Ch...,papakichi111
4,2022-12-01 23:59:20+00:00,1598466812198387712,I like ChatGPT as much as the next guy but th...,DennisLibre
...,...,...,...,...
5663206,2022-11-09 00:00:50+00:00,1590132269255114753,Reduce cost and improve query performance with...,tweepy20220507
5663207,2022-11-09 00:00:36+00:00,1590132210715303936,Enable cross account queries on AWS CloudTrail...,tweepy20220507
5663208,2022-11-09 00:00:12+00:00,1590132106260353024,If you want to generate more leads create con...,PirenneSalvor
5663209,2022-11-09 00:00:03+00:00,1590132069069619200,AnnaTynan Yes bc we share an inbox I am lu...,ceciliaclyra


In [14]:
def decode_text(text):
    decoded_text = text.encode('ascii', 'ignore').decode('utf-8')

    # decode HTML entities
    decoded_html = html.unescape(decoded_text)
    return ''.join([word for word in decoded_html if word.isprintable()])

def remove_mentions(text):
    return re.sub("@[A-Za-z0-9_]+","", text)

def remove_stopwords(words_list):
    stop_list = stopwords.words("english")
    stop_list.append("filler")
    stop_list.extend(["html", "http", "https", "www", "gt", "amp", "com", "nbsp", "em", "en", "lt", "quot"])
    clean_sentence = ''
    for word in words_list:
        if word not in stop_list:
            clean_sentence += word + ' '
    return clean_sentence

def clean_original_text(text):
    text = str(text)
    text = text.lower()
    decoded_sentence = decode_text(text)
    decoded_tokens = decoded_sentence.split()
    clean_sentence = remove_stopwords(decoded_tokens)

    return clean_sentence

In [15]:
concat_df['clean_text'] = concat_df['text'].apply(clean_original_text)

In [16]:
concat_df.to_csv("../Temp/preprocessed.csv", index=False)

# Sentiment Analysis

In [2]:
preprocessed_df = pd.read_csv("../Temp/preprocessed.csv")

c:\Users\jewel\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
preprocessed_df.isna().sum()

datetime          0
id                0
text             51
username          0
clean_text    39481
dtype: int64

In [4]:
preprocessed_df.dropna(inplace=True)

In [5]:
preprocessed_df.head()

,datetime,id,text,username,clean_text
0,2022-12-01 23:59:58+00:00,1598466969141137408,The new ChatGPT model by OpenAI is astonishi...,askviable,new chatgpt model openai astonishing humanizes...
1,2022-12-01 23:59:56+00:00,1598466963466252288,bitquark I am looking forward for a ChatGPT A...,mazen160,bitquark looking forward chatgpt api see lot t...
2,2022-12-01 23:59:37+00:00,1598466881761218560,i think i broke chatGPT it s been frozen for ...,ZoumanaCisse6,think broke chatgpt frozen couple minutes co h...
3,2022-12-01 23:59:26+00:00,1598466834776600576,YanoTomo aizu sniper yae AI Ch...,papakichi111,yanotomo aizu sniper yae ai chatgpt worldcup c...
4,2022-12-01 23:59:20+00:00,1598466812198387712,I like ChatGPT as much as the next guy but th...,DennisLibre,like chatgpt much next guy nothing else twitte...


In [6]:
new_words = {
        'easy-to-use': 10,
        'productive': 5,
        'slow': -5,
        'frustrating': -10,
        'glitchy': -100,
}

# Instantiate the sentiment intensity analyzer with the existing lexicon
vader = SentimentIntensityAnalyzer()
# Update the lexicon
vader.lexicon.update(new_words)
# Iterate through the headlines and get the polarity scores
scores = preprocessed_df['clean_text'].apply(vader.polarity_scores)


In [7]:
scores_df = pd.DataFrame.from_records(scores)


In [8]:
scored_tweets = preprocessed_df.join(scores_df)

In [9]:
scored_tweets['datetime'] = pd.to_datetime(scored_tweets['datetime'], utc=True)


In [28]:
scored_tweets

,datetime,id,text,username,clean_text,neg,neu,pos,compound,sentiment
0,2022-12-01 23:59:58+00:00,1598466969141137408,The new ChatGPT model by OpenAI is astonishi...,askviable,new chatgpt model openai astonishing humanizes...,0.000,1.000,0.000,0.0000,neutral
1,2022-12-01 23:59:56+00:00,1598466963466252288,bitquark I am looking forward for a ChatGPT A...,mazen160,bitquark looking forward chatgpt api see lot t...,0.000,1.000,0.000,0.0000,neutral
2,2022-12-01 23:59:37+00:00,1598466881761218560,i think i broke chatGPT it s been frozen for ...,ZoumanaCisse6,think broke chatgpt frozen couple minutes co h...,0.237,0.763,0.000,-0.4215,negative
3,2022-12-01 23:59:26+00:00,1598466834776600576,YanoTomo aizu sniper yae AI Ch...,papakichi111,yanotomo aizu sniper yae ai chatgpt worldcup c...,0.000,1.000,0.000,0.0000,neutral
4,2022-12-01 23:59:20+00:00,1598466812198387712,I like ChatGPT as much as the next guy but th...,DennisLibre,like chatgpt much next guy nothing else twitte...,0.000,0.878,0.122,0.3612,positive
...,...,...,...,...,...,...,...,...,...,...
5663206,2022-11-09 00:00:50+00:00,1590132269255114753,Reduce cost and improve query performance with...,tweepy20220507,reduce cost improve query performance amazon a...,NaN,NaN,NaN,NaN,positive
5663207,2022-11-09 00:00:36+00:00,1590132210715303936,Enable cross account queries on AWS CloudTrail...,tweepy20220507,enable cross account queries aws cloudtrail la...,NaN,NaN,NaN,NaN,positive
5663208,2022-11-09 00:00:12+00:00,1590132106260353024,If you want to generate more leads create con...,PirenneSalvor,want generate leads create content answers cus...,NaN,NaN,NaN,NaN,positive
5663209,2022-11-09 00:00:03+00:00,1590132069069619200,AnnaTynan Yes bc we share an inbox I am lu...,ceciliaclyra,annatynan yes bc share inbox lucky get see que...,NaN,NaN,NaN,NaN,positive


In [10]:
def label_sentiment(score):
    if score < 0:
        return "negative"
    elif score == 0:
        return "neutral"
    else:
        return "positive"

In [11]:
scored_tweets['sentiment'] = scored_tweets['compound'].apply(lambda score: label_sentiment(score))

In [12]:
scored_tweets_copy = scored_tweets.copy()
scored_tweets_copy.dropna(inplace=True)

In [13]:
scored_tweets_copy.to_csv("../Temp/sentiment.csv")

In [14]:
scored_tweets_copy.set_index('datetime', inplace=True)

# Convert index to timezone-naive
scored_tweets_copy.index = scored_tweets_copy.index.tz_localize(None)

# Calculate daily averages and create new dataframe
df_daily = scored_tweets_copy.resample('D').mean()

# Add 'Date' column to new dataframe
df_daily['Date'] = df_daily.index.date

# Reset index (index is still timezone-naive)
df_daily.reset_index(drop=True, inplace=True)

# Reorder columns
df_daily = df_daily[['Date', 'neg', 'neu', 'pos', 'compound']]
df_daily["Date"] = df_daily["Date"].astype(str)

In [15]:
df_daily.to_json('../Temp/daily_score.json', orient='records', indent=4)

In [16]:
AWS_ACCESS_KEY_ID=AWS_ACCESS_KEY_ID
AWS_SECRET_ACCESS_KEY=AWS_SECRET_ACCESS_KEY
BUCKET_NAME = BUCKET_NAME

s3 = boto3.resource('s3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name='us-east-1')

BUCKET = s3.Bucket(BUCKET_NAME)

In [29]:
with open('../Temp/daily_score.json', 'rb') as file:
    s3_path = '{}/{}'.format('sentiment', 'daily_score.json')
    BUCKET.put_object(Key=s3_path, Body=file)

# Topic Modeling

In [2]:
senti_df = pd.read_csv("../Temp/sentiment.csv")

c:\Users\jewel\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
senti_df.isna().sum()

Unnamed: 0    0
datetime      0
id            0
text          0
username      0
clean_text    0
neg           0
neu           0
pos           0
compound      0
sentiment     0
dtype: int64

In [4]:
try:
    senti_df.drop('Unnamed: 0', axis=1, inplace=True)
except:
    senti_df = senti_df

In [5]:
grouped_df = senti_df.groupby("sentiment")

In [6]:
nltk_words = words.words()

In [7]:
def get_sentiment_topic_terms(grouped_df):
    sentiment_topic_terms = {}

    for sentiment, sentiment_df in grouped_df:
        print(sentiment)
        topic_terms = generate_sentiment_topic_model(sentiment_df)
        sentiment_topic_terms[sentiment] = topic_terms 
        
    return sentiment_topic_terms
    
def generate_sentiment_topic_model(df):
    chatgptreddit_vecs, chatgptreddit_dict = create_lda_vecs(df)
    topic_terms_dict = get_topic_terms(chatgptreddit_vecs, chatgptreddit_dict)
    
    return topic_terms_dict

""" Functions for LDA topic modelling """
def docs2vecs(docs, dictionary):
    # docs is a list of documents returned by corpus2docs.
    # dictionary is a gensim.corpora.Dictionary object.
    vecs = [dictionary.doc2bow(doc) for doc in docs]
    return vecs

def create_lda_vecs(df):
    chatgptreddit_docs = []
    for doc in df["clean_text"]:
        try:
            tokens = doc.split()  # split the sentence into a list of tokens
            tokens = [token for token in tokens]
            chatgptreddit_docs.append(tokens)
        except:
            continue
    chatgptreddit_dict = gensim.corpora.Dictionary(chatgptreddit_docs)
    chatgptreddit_vecs = docs2vecs(chatgptreddit_docs, chatgptreddit_dict)
    return chatgptreddit_vecs, chatgptreddit_dict

def get_topic_terms(chatgptreddit_vecs, chatgptreddit_dict):
    chatgptreddit_lda = gensim.models.ldamodel.LdaModel(corpus=chatgptreddit_vecs, id2word=chatgptreddit_dict, num_topics=2)
    topic_terms_dict = defaultdict(list)
    for topic_id, terms_list in chatgptreddit_lda.show_topics(num_topics=2, num_words=10, formatted=False):
        terms = []
        for term in terms_list:
            # if no variations of the word are found
            if len(term[0]) > 1 and term[0] in nltk_words:
                terms.append((term[0].lower(), term[1]))
        sorted_terms = sorted(terms, key=lambda x: x[1], reverse=True)
        topic_terms_dict["Topic " + str(topic_id)] = sorted_terms
    topic_terms_dict = dict(topic_terms_dict)
    return topic_terms_dict

In [10]:
sentiment_topic_terms = get_sentiment_topic_terms(grouped_df)

negative
neutral
positive


In [11]:
with open('../Temp/sentiment_topics.json', "w") as new_file:
    output_dict = {}
    for sentiment, topics in sentiment_topic_terms.items():
        topic_dict = {}
        for topic_id, terms in topics.items():
            term_list = []
            for term in terms:
                print()
                term_dict = {"word": term[0], "frequency": round(float(term[1]), 4) }
                term_list.append(term_dict)
            topic_dict[topic_id] = term_list
        output_dict[sentiment] = topic_dict
    json.dump(output_dict, new_file, indent=4)

In [12]:
AWS_ACCESS_KEY_ID=AWS_ACCESS_KEY_ID
AWS_SECRET_ACCESS_KEY=AWS_SECRET_ACCESS_KEY
BUCKET_NAME = BUCKET_NAME

s3 = boto3.resource('s3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name='us-east-1')

BUCKET = s3.Bucket(BUCKET_NAME)

In [13]:
with open('../Temp/sentiment_topics.json', 'rb') as file:
    s3_path = '{}/{}'.format('topic', 'sentiment_topics.json')
    BUCKET.put_object(Key=s3_path, Body=file)
print('Completed')